In [1]:
# Import stuff
import pandas as pd
import numpy as np

In [2]:
# # Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# # ms-python.python added
# import os
# try:
# 	# os.chdir(os.path.join(os.getcwd(), '..'))
#     os.chdir(os.path.join(os.getcwd(), "./scripts-notebooks"))
#     print(os.getcwd())
# except Exception as e:
# 	print(e)

/Users/SamYang95/GRN_Inference_IY/scripts-notebooks


In [3]:
# Get expression data
ko_df = pd.read_csv('../dataset/yeast_networks/expression/KO.txt', sep='\t').set_index('Gene')
nv_df = pd.read_csv('../dataset/yeast_networks/expression/NatVar.txt', sep='\t').set_index('Gene')
stress_df = pd.read_csv('../dataset/yeast_networks/expression/Stress.txt', sep='\t').set_index('Name')

# Get Transcription Factors
ko_tf = pd.read_csv('../dataset/yeast_networks/expression/KO_TF_names.txt', sep='\t', header=None)
ko_tf.columns = ['TF']

nv_tf = pd.read_csv('../dataset/yeast_networks/expression/NatVar_TF_names.txt', sep='\t', header=None)
nv_tf.columns = ['TF']

stress_tf = pd.read_csv('../dataset/yeast_networks/expression/Stress_TF_names.txt', sep='\t', header=None)
stress_tf.columns = ['TF']

# Note that there are different gene list for each df, so we will use ko as test for now.

In [26]:
def merge_dataset(exp_df_list, merge_method):
    if len(exp_df_list) == 0: raise Exception("input list cannot be empty")

    exp_df = exp_df_list[0]
    for i in range(1, len(exp_df_list)):
        exp_df = merge_method(exp_df, exp_df_list[i])
        # exp_df = exp_df.merge(exp_df_list[i], how='inner', left_index=True, right_index=True)

    return exp_df

In [31]:
merge_method_df = lambda x, y: x.merge(y, how='inner', left_index=True, right_index=True)
merge_method_tf = lambda x, y: x.merge(y, how='inner', left_on='TF', right_on='TF')

In [32]:
df_all = merge_dataset([ko_df, nv_df, stress_df], merge_method=merge_method_df)

In [33]:
tf_all = merge_dataset([ko_tf, nv_tf, stress_tf], merge_method=merge_method_tf)

In [34]:
df_all.to_csv('../dataset/yeast_networks/expression/All.txt', sep='\t')